# Importing libraries, loading and transforming data

In [32]:
!pip install -q evaluate transformers==4.28.1
!pip install -U -q datasets
!pip install -q torchaudio==0.12
!add-apt-repository -y ppa:savoury1/ffmpeg4 
!apt-get -qq install -y ffmpeg
!pip install -q mlflow

ERROR: Ignored the following yanked versions: 2.0.0
ERROR: Could not find a version that satisfies the requirement torchaudio==0.12 (from versions: 2.0.1, 2.0.2, 2.1.0, 2.1.1, 2.1.2)
ERROR: No matching distribution found for torchaudio==0.12
'add-apt-repository' is not recognized as an internal or external command,
operable program or batch file.
'apt-get' is not recognized as an internal or external command,
operable program or batch file.


In [31]:
#imports
import pandas as pd
import gc
import re
import numpy as np
import torch
from imblearn.over_sampling import RandomOverSampler
import datasets
import transformers
print(transformers.__version__)

import warnings 
warnings.filterwarnings("ignore")

from tqdm import tqdm
tqdm.pandas()

ModuleNotFoundError: No module named 'torch'

In [ ]:
RATE_HZ = 16000 # resampling rate in Hz
MAX_LENGTH = 480000 # maximum audio interval length to consider (= RATE_HZ * SECONDS)
labels = ['disco', 'metal', 'reggae', 'blues', 'rock', 'classical', 'jazz', 'hiphop', 'country', 'pop']
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

print(id2label, '\n\n', label2id)

# Load and preprocess data

In [ ]:
from pathlib import Path
import torchaudio

def load_data():
    file_list = []
    label_list = []
    for file in Path('/kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original/').glob('*/*.wav'):
        if not 'jazz.00054' in str(file): # skip bad example
            genre = re.match(r"(\D+)", file.stem).group(1).rstrip('.')
            file_list.append(file)
            label_list.append(label2id[genre])
    dd = pd.DataFrame()
    dd['file'] = file_list
    dd['label'] = label_list
    return dd

In [ ]:
%%time
dd = load_data()
dd.head()

In [ ]:
# random oversampling of minority class # ensure that each class has the same number of records
y = dd[['label']]
dd = dd.drop(['label'], axis=1)
ros = RandomOverSampler(random_state=83)
dd, y_resampled = ros.fit_resample(dd, y)
del y
dd['label'] = y_resampled
del y_resampled
gc.collect()

In [ ]:
dd.shape, dd['label'].value_counts()

In [ ]:
def get_transform_audio(file):
    audio,rate = torchaudio.load(str(file))
    transform = torchaudio.transforms.Resample(rate,RATE_HZ)
    audio = transform(audio).squeeze(0).numpy()
    audio = audio[:MAX_LENGTH]
    return audio # truncate to first part of audio to save RAM
dd['audio'] = dd['file'].progress_apply(get_transform_audio)

In [ ]:
%%time
dd = dd.drop(['file'], axis=1)

In [ ]:
dd.sample(5)

In [ ]:
from datasets import Dataset, ClassLabel
dd = Dataset.from_pandas(dd)

In [ ]:
from collections import Counter
Counter(dd['label']).items()

In [ ]:
dd = dd.train_test_split(test_size=0.2)
dd

# Load facebook/wav2vec2-base-960h model

In [ ]:
from transformers import AutoFeatureExtractor, AutoModelForAudioClassification

model_str = "facebook/wav2vec2-base-960h" 
feature_extractor = AutoFeatureExtractor.from_pretrained(model_str)
model = AutoModelForAudioClassification.from_pretrained(model_str,num_labels=len(labels))
model.config.id2label = id2label
# number of trainable parameters
print(model.num_parameters(only_trainable=True)/1e6)

In [ ]:
def preprocess_function(batch):    
    inputs = feature_extractor(batch['audio'], sampling_rate=RATE_HZ, max_length=MAX_LENGTH, truncation=True)
    inputs['input_values'] = inputs['input_values'][0]
    return inputs

dd['test'] = dd['test'].map(preprocess_function, remove_columns="audio", batched=False)
dd['train'] = dd['train'].map(preprocess_function, remove_columns="audio", batched=False)

In [ ]:
gc.collect()

# Train and evaluate model

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

from sklearn.metrics import roc_auc_score
def compute_metrics(eval_pred):
    # Compute the ROC AUC score
    predictions = eval_pred.predictions
    predictions = np.exp(predictions)/np.exp(predictions).sum(axis=1, keepdims=True)
    label_ids = eval_pred.label_ids
    roc_auc = roc_auc_score(label_ids, predictions, average='macro', multi_class='ovr') # one-vs-rest ROC AUC score
    
    # Calculate accuracy using the loaded accuracy metric
    acc_score = accuracy.compute(predictions=predictions.argmax(axis=1), references=label_ids)['accuracy']
    
    return {
        "roc_auc": roc_auc,
        "accuracy": acc_score
    }

In [ ]:
from transformers import TrainingArguments, Trainer
batch_size=4
warmup_steps=50
weight_decay=0.02
num_train_epochs=25
model_name = "music_genres_classification"
training_args = TrainingArguments(
    output_dir=model_name,
    logging_dir='./logs',
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=1e-5, # 5e-6
    logging_strategy='steps',
    logging_first_step=True,
    load_best_model_at_end=True,
    logging_steps=1,
    evaluation_strategy='epoch',
    warmup_steps=warmup_steps,
    weight_decay=weight_decay,
    eval_steps=1,
    gradient_accumulation_steps=1, 
    gradient_checkpointing=True,
    save_strategy='epoch',
    save_total_limit=1, # save fewer checkpoints to limit used space
    report_to="mlflow",  # log to mlflow
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dd["train"],
    eval_dataset=dd["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.evaluate()

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
trainer.save_model()

In [ ]:
from transformers import pipeline

pipe=pipeline('audio-classification',model=model_name,device=0)

In [ ]:
# disco example
audio,rate=torchaudio.load('/kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original/disco/disco.00005.wav')
transform=torchaudio.transforms.Resample(rate,RATE_HZ)
audio=transform(audio).numpy().reshape(-1)
# make a classification pipeline
pipe(audio)

In [ ]:
# classical example
audio,rate=torchaudio.load('/kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original/classical/classical.00014.wav')
transform=torchaudio.transforms.Resample(rate,RATE_HZ)
audio=transform(audio).numpy().reshape(-1)
# make a classification pipeline
pipe(audio)

In [ ]:
from IPython.display import Audio
Audio(audio,rate=RATE_HZ)

# Send model to Huggingface

In [ ]:
# finally, save the model to Huggingface
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from huggingface_hub import HfApi
api = HfApi()
repo_id = f"dima806/{model_name}"
try:
    api.create_repo(repo_id)
except:
    print(f"Repo {repo_id} already exists")

In [ ]:
api.upload_folder(
    folder_path=model_name,
    path_in_repo = ".",
    repo_id=repo_id,
    repo_type="model"
)

### Finally, the model is saved and can be used from https://huggingface.co/dima806/music_genres_classification